#Extract EXIF TAGS 

In [1]:
import json
import os

from PIL import Image
from PIL.ExifTags import TAGS


In [2]:
def decodeDictionary(dictionary):
  if type(dictionary) == dict:
    for key in dictionary.keys():
        dictionary[key] = decodeDictionary(dictionary[key])
  elif type(dictionary) == bytes:
        dictionary = dictionary.decode('ISO-8859-1') 
  return dictionary

In [3]:

pasta = "/content/drive/MyDrive/Colab Notebooks/PDI - APS/ft-DB/"
data = {} 
for diretorio, subpastas, arquivos in os.walk(pasta):

    for arquivo in arquivos:
        if (arquivo.split('.')[-1].upper() in ('JPG', 'PNG')) & (os.path.isfile(os.path.join(diretorio, arquivo))):

            image = Image.open(os.path.join(diretorio, arquivo))

            exifdata = image.getexif()

            meta = {}
            meta["Image Size"] = image.size,
            meta["Image Height"] = image.height,
            meta["Image Width"] = image.width,
            meta["Image Format"] = image.format,
            meta["Image Mode"] = image.mode,
           
            meta["Image is Animated"] = getattr(image, "is_animated", False),
            meta[ "Frames in Image"] = getattr(image, "n_frames", 1)

            for tagid in exifdata:

                tagname = TAGS.get(tagid, tagid)

                value = exifdata.get(tagid)

                meta[tagname] = value if type(value) not in (bytes, dict) else decodeDictionary(value)

            data[os.path.join(diretorio.replace(pasta, ""), arquivo)] =  meta


In [4]:
with open('metadata.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)